In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

amazon
digits
titanic



In [2]:
# coding: utf-8
# import data
import os
import matplotlib.image as image
import matplotlib.pyplot as plt

PLANET_KAGGLE_ROOT = "../input/amazon"
PLANET_KAGGLE_TIFF_DIR = PLANET_KAGGLE_ROOT + "/train-tif/"
PLANET_KAGGLE_LABEL_CSV = PLANET_KAGGLE_ROOT + "/train.csv"

In [3]:
# load label data
# read our data and take a look at what we are dealing with
train_csv = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
print(train_csv.head())

  image_name                                       tags
0    train_0                               haze primary
1    train_1            agriculture clear primary water
2    train_2                              clear primary
3    train_3                              clear primary
4    train_4  agriculture clear habitation primary road


In [4]:
# load tiff data
# load files
train_tif = os.listdir(PLANET_KAGGLE_TIFF_DIR)
len(train_tif)

40479

In [5]:
# pick just cloudy images for now

AVAILABLE_LABELS = [
    'agriculture', 
    'artisinal_mine', 
    'bare_ground', 
    'blooming', 
    'blow_down', 
    'clear', 
    'cloudy', 
    'conventional_mine', 
    'cultivation', 
    'habitation', 
    'haze', 
    'partly_cloudy', 
    'primary', 
    'road', 
    'selective_logging', 
    'slash_burn', 
    'water']

tags = pd.DataFrame()

for label in AVAILABLE_LABELS:
    tags[label] = train_csv.tags.apply(lambda x: np.where(label in x, 1, 0))

train = pd.concat([train_csv, tags], axis=1)
train.head()

,image_name,tags,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,train_0,haze primary,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,train_1,agriculture clear primary water,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,train_2,clear primary,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,train_3,clear primary,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,train_4,agriculture clear habitation primary road,1,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0


In [6]:
#filtered_samples = train.sample(n=1000)

cloudy_samples = train.loc[train.cloudy == 1].sample(n=1000)
cloudy_samples.head()

,image_name,tags,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
28662,train_28662,partly_cloudy primary,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0
17,train_17,partly_cloudy primary,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0
23154,train_23154,partly_cloudy primary,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0
22959,train_22959,partly_cloudy primary,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0
10265,train_10265,cloudy,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
non_cloudy_samples = train.loc[train.cloudy == 0].sample(n=1000)
non_cloudy_samples.head()

,image_name,tags,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
2674,train_2674,clear primary,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
22416,train_22416,agriculture clear cultivation primary road,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0
7283,train_7283,clear primary,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
39227,train_39227,clear primary water,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
20323,train_20323,agriculture clear cultivation primary,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0


In [8]:
all_cloudy_samples = pd.concat([cloudy_samples, non_cloudy_samples])[['image_name', 'cloudy']]

from sklearn.utils import shuffle

all_cloudy_samples = shuffle(all_cloudy_samples)
all_cloudy_samples.head()

,image_name,cloudy
25962,train_25962,0
12052,train_12052,1
39531,train_39531,0
3938,train_3938,0
8294,train_8294,0


In [9]:
from glob import glob
from skimage import io

image_paths = sorted([glob('../input/amazon/train-tif/'+ image +'.tif') for image in all_cloudy_samples['image_name']])
imgs = [io.imread(path[0]) for path in image_paths]

In [10]:
flattened_imgs = [img.flatten() for img in imgs]
flattened_imgs[0]

array([4074, 3435, 2183, ..., 3515, 2239, 7118], dtype=uint16)

In [ ]:
df = pd.DataFrame.from_records(flattened_imgs)

In [ ]:
df = pd.concat([all_cloudy_samples[['cloudy']], df], axis=1)
df

In [ ]:
# create test/training set

from sklearn import preprocessing, cross_validation 

X = df.ix[:, 1:].as_matrix()
y = df.cloudy.as_matrix();


[X_train, X_test, y_train, y_test] = cross_validation.train_test_split(X, y, test_size = 0.2)

print('training sets created')

In [ ]:
X_train[0]

In [ ]:
# create neural network model, one vs all

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(100,), verbose=True)

clf = clf.fit(X_train, y_train);

print("neural network model trained")

In [ ]:
# accuracy of linear regression model: 
accuracy = clf.score(X_test, y_test)

print(accuracy)